In [81]:
from functools import reduce
import operator

class CyclicList(list):
    def __init__(self, content):
        super(CyclicList, self).__init__(content)
        
    def __getitem__(self, index):
        return super(CyclicList, self).__getitem__(index % len(self))
    
    def __setitem__(self, index, value):
        return super(CyclicList, self).__setitem__(index % len(self), value)


def fold(length, lengths):
    values = CyclicList(range(length))
    skip_size = 0
    position = 0
    for length in lengths:
        for i in range(int(length / 2)):
            (values[position+i], values[position+length-i-1]) = (values[position+length-i-1], values[position+i])
        position += length + skip_size
        skip_size += 1
    return values


def make_indices(string, rounds):
    for r in range(rounds):
        for c in string:
            yield ord(c)
        for s in [17, 31, 73, 47, 23]:
            yield s

            
def densify(values):
    v = list(values)
    return [reduce(operator.xor, v[i:i+16], 0) for i in range(0, len(v), 16)]


def hash_string(string):
    values = fold(256, make_indices(string, 64))
    return densify(values)


def binary_hash(string):
    return [bool(byte & (1 << o)) for byte in hash_string(string) for o in reversed(range(8))]

In [82]:
import numpy as np
def hamming_weight(byte):
    return sum(bool(byte & (1 << o)) for o in range(8))

def count_ones(key):
    return sum(hamming_weight(d) for i in range(128) for d in  hash_string("{}-{}".format(key, i)))

def count_ones_binary(key):
    return sum(sum(binary_hash("{}-{}".format(key, i))) for i in range(128))

def make_map(key):
    return np.array([binary_hash("{}-{}".format(key, i)) for i in range(128)])

def print_map(m):
    for row in m:
        print(''.join('#' if e else '.' for e in row))

In [83]:
map1 = make_map('flqrgnkx')
map2 = make_map('ffayrhll')

In [86]:
print(map1[0:8, 0:8])

[[ True  True False  True False  True False False]
 [False  True False  True False  True False  True]
 [False False False False  True False  True False]
 [ True False  True False  True  True False  True]
 [False  True  True False  True False False False]
 [ True  True False False  True False False  True]
 [False  True False False False  True False False]
 [ True  True False  True False  True  True False]]


In [88]:
import numpy as np

def neighbors(position):
    offsets = [
        np.array((1, 0)),
        np.array((-1, 0)),
        np.array((0, 1)),
        np.array((0, -1))
    ]
    for o in offsets:
        n = position + o
        if all(n >= 0) and all(n < 128):
            yield n

def fill_region(m, position):
    elements = [position]
    while elements:
        p = elements.pop()
        if m[tuple(p)]:
            for n in neighbors(p):
                elements.append(n)
            m[tuple(p)] = False

def count_regions(m):
    count = 0
    m = m.copy()
    for r in range(len(m)):
        for c in range(len(m[r])):
            if m[r, c]:
                fill_region(m, np.array((r, c)))
                count += 1
    return count
count_regions(map2)

1134